In [4]:
import pandas as pd
import numpy as np

# Read in county-level results
* Number of votes
* North Dakota is for the whole state

In [5]:
alabama = 'data/alabama_results.xlsx'
arkansas = 'data/arkansas_results.xlsx'
california = 'data/california_results.xlsx'
colorado = 'data/colorado_results.xlsx'
idaho = 'data/idaho_results.xlsx'
iowa = 'data/iowa_results.xlsx'
maine = 'data/maine_results.xlsx'
massachusetts = 'data/massachusetts_results.xlsx'
michigan = 'data/michigan_results.xlsx'
minnesota = 'data/minnesota_results.xlsx'
mississippi = 'data/mississippi_results.xlsx'
missouri = 'data/missouri_results.xlsx'
nevada = 'data/nevada_results.xlsx'
new_hampshire = 'data/new_hampshire_results.xlsx'
north_carolina = 'data/north-carolina_results.xlsx'
north_dakota = 'data/north-dakota_results.xlsx'
oklahoma = 'data/oklahoma_results.xlsx'
south_carolina = 'data/south_carolina_results.xlsx'
tennessee = 'data/tennessee_results.xlsx'
texas = 'data/texas_results.xlsx'
utah = 'data/utah_results.xlsx'
vermont = 'data/vermont_results.xlsx'
virginia = 'data/virginia_results.xlsx'
washington = 'data/washington_results.xlsx'

In [6]:
# create list  containing states and state abbreviations
state_list = [[alabama, 'AL'],
              [arkansas, 'AR'],
              [california, 'CA'],
              [colorado, 'CO'],
              [idaho, 'ID'],
              [iowa, 'IA'],
              [maine, 'ME'],
              [massachusetts, 'MA'],
              [michigan, 'MI'],
              [minnesota, 'MN'],
              [mississippi, 'MS'],
              [missouri, 'MO'],
              [nevada, 'NV'],
              [new_hampshire, 'NH'],
              [north_carolina, 'NC'],
              [oklahoma, 'OK'],
              [south_carolina, 'SC'],
              [tennessee, 'TN'],
              [texas, 'TX'],
              [utah, 'UT'],
              [vermont, 'VT'],
              [virginia, 'VA'],
              [washington, 'WA']]



# Combine Election Results
* Results are combined into one dataframe with the state abbreviation added on

In [7]:
# function for combining all the state results
def combine_results(state_list):
    
    global all_results
    
    all_results = pd.DataFrame()
    
    for state in state_list:
        results = pd.read_excel(state[0])
        active_candidates_LN = ['Sanders', 'Biden']
        
        results_df = results.transpose().reset_index()
        results_df.columns = results_df.iloc[0]
        results_df = results_df.drop(results_df.index[0])
        results_df = results_df.melt(id_vars=['Counties'],value_vars=results_df.columns.to_list()[1:])
        results_df['State'] = state[1]
        results_df = results_df[results_df['Counties'].isin(active_candidates_LN)]
        results_df = results_df.sort_values(by=[0, 'value'], ascending=False)

        results_df.rename(columns={"Counties": "last name", 0: "county", "State": "state", 'value': 'votes'}, inplace=True)
        
        all_results = all_results.append(results_df)

combine_results(state_list)

# add north dakota as a county
all_results.loc[978] = ['Biden', 'North Dakota', 5742, 'ND']
all_results.loc[979] = ['Sanders', 'North Dakota', 7682, 'ND']

all_results

,last name,county,votes,state
991,Biden,Winston,266,AL
990,Sanders,Winston,117,AL
976,Biden,Wilcox,1864,AL
975,Sanders,Wilcox,284,AL
961,Biden,Washington,1109,AL
...,...,...,...,...
13,Sanders,Asotin,705,WA
0,Sanders,Adams,288,WA
1,Biden,Adams,284,WA
978,Biden,North Dakota,5742,ND


# Assign county winner and percent votes classification column

In [8]:
all_results_winners = pd.DataFrame()

for index,row in all_results[['state','county']].drop_duplicates().iterrows():
    
    # filter for unique counties
    filtered_df = all_results[all_results['state'] == row['state']]
    filtered_df = filtered_df[filtered_df['county'] == row['county']]
    
    # get total number of votes for comparison
    vote_sum = filtered_df['votes'].sum()
    
    winner = []
    percent_votes = []
    
    for index,row in filtered_df.iterrows():
        if row['votes'] >= (vote_sum * 0.5):
            winner.append(1)
        else:
            winner.append(0)
        
        if vote_sum > 0:
            percent_votes.append(row['votes'] / vote_sum)
        else: percent_votes.append(0)
            
    filtered_df['winner'] = winner
    filtered_df['percent_votes'] = percent_votes
    
    all_results_winners = all_results_winners.append(filtered_df)
    
all_results_winners

,last name,county,votes,state,winner,percent_votes
991,Biden,Winston,266,AL,1,0.694517
990,Sanders,Winston,117,AL,0,0.305483
976,Biden,Wilcox,1864,AL,1,0.867784
975,Sanders,Wilcox,284,AL,0,0.132216
961,Biden,Washington,1109,AL,1,0.876680
...,...,...,...,...,...,...
13,Sanders,Asotin,705,WA,0,0.386725
0,Sanders,Adams,288,WA,1,0.503497
1,Biden,Adams,284,WA,0,0.496503
978,Biden,North Dakota,5742,ND,0,0.427741


# Assign state winner classification column

In [9]:
all_results_winners_state = pd.DataFrame()

for index,row in all_results_winners[['state']].drop_duplicates().iterrows():
    
    # filter for unique state
    filtered_df = all_results_winners[all_results_winners['state'] == row['state']]
    
    # get total number of votes for comparison
    winner_sum = filtered_df['winner'].sum()
    
    winner = []
    
    biden_sum = filtered_df[filtered_df['last name'] == 'Biden']['winner'].sum()
    
    sanders_sum = filtered_df[filtered_df['last name'] == 'Sanders']['winner'].sum()

    for index,row in filtered_df.iterrows():
        if row['last name'] == 'Biden':
            if biden_sum > sanders_sum:
                winner.append(1)
            else:
                winner.append(0)    
        else:
            if biden_sum > sanders_sum:
                winner.append(0)
            else:
                winner.append(1)
                
    filtered_df['winner_state'] = winner
    
    
    
    all_results_winners_state = all_results_winners_state.append(filtered_df)

all_results_winners_state

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,last name,county,votes,state,winner,percent_votes,winner_state
991,Biden,Winston,266,AL,1,0.694517,1
990,Sanders,Winston,117,AL,0,0.305483,0
976,Biden,Wilcox,1864,AL,1,0.867784,1
975,Sanders,Wilcox,284,AL,0,0.132216,0
961,Biden,Washington,1109,AL,1,0.876680,1
...,...,...,...,...,...,...,...
13,Sanders,Asotin,705,WA,0,0.386725,0
0,Sanders,Adams,288,WA,1,0.503497,0
1,Biden,Adams,284,WA,0,0.496503,1
978,Biden,North Dakota,5742,ND,0,0.427741,0


# Import county facts
* A few of the columns are renamed

In [10]:
county_facts = pd.read_csv('data/county_facts.csv')
county_facts = county_facts.rename(columns={'state_abbreviation': 'state', 'area_name': 'county'})

# Add a merge column
* Add a merging column to results and county facts dataframes
* This column standardizes the county name to make merging easier

In [23]:
county_merge = []

for index,row in all_results_winners_state.iterrows():
    
    split_county = []
    county_name = ''
    
    split_county = row['county'].lower().split()

    if 'county' in split_county:
        split_county.remove('county')
    
    county_name = ''.join(split_county)
    
    county_merge.append(county_name)

all_results_winners_state['county_merge'] = county_merge

all_results_winners_state['state'].drop_duplicates()

991     AL
1407    AR
1026    CA
1009    CO
732     ID
1339    IA
196     ME
27      MA
1067    MI
1291    MN
892     MS
2167    MO
192     NV
18      NH
1486    NC
1141    OK
495     SC
1411    TN
3795    TX
420     UT
26      VT
1981    VA
494     WA
978     ND
Name: state, dtype: object

* for county facts

In [14]:
county_facts['county_merge'] = ''

for index,row in county_facts.iterrows():
    
    split_county = []
    county_name = ''
    
    split_county = row['county'].lower().split()

    if 'county' in split_county:
        split_county.remove('county')
    
    county_name = ''.join(split_county)

    county_facts.loc[index, 'county_merge'] = county_name

# Merge county facts and results

In [29]:
merged = pd.merge(all_results_winners_state, county_facts, how='left', on=['county_merge', 'state'])

merged

,last name,county_x,votes,state,winner,percent_votes,winner_state,county_merge,fips,county_y,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
0,Biden,Winston,266,AL,1,0.694517,1,winston,1133.0,Winston County,...,0.0,19.6,797845.0,165638.0,166709.0,6853.0,16564.0,7.0,612.98,39.9
1,Sanders,Winston,117,AL,0,0.305483,0,winston,1133.0,Winston County,...,0.0,19.6,797845.0,165638.0,166709.0,6853.0,16564.0,7.0,612.98,39.9
2,Biden,Wilcox,1864,AL,1,0.867784,1,wilcox,1131.0,Wilcox County,...,0.0,37.6,0.0,21337.0,75708.0,6018.0,12949.0,0.0,888.50,13.1
3,Sanders,Wilcox,284,AL,0,0.132216,0,wilcox,1131.0,Wilcox County,...,0.0,37.6,0.0,21337.0,75708.0,6018.0,12949.0,0.0,888.50,13.1
4,Biden,Washington,1109,AL,1,0.876680,1,washington,1129.0,Washington County,...,0.0,25.6,1153449.0,32856.0,76343.0,4458.0,0.0,2.0,1080.21,16.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3243,Sanders,Asotin,705,WA,0,0.386725,0,asotin,53003.0,Asotin County,...,0.0,17.0,0.0,0.0,208856.0,9867.0,22983.0,35.0,636.21,34.0
3244,Sanders,Adams,288,WA,1,0.503497,0,adams,53001.0,Adams County,...,16.3,0.0,442178.0,574216.0,144809.0,8547.0,14844.0,50.0,1924.98,9.7
3245,Biden,Adams,284,WA,0,0.496503,1,adams,53001.0,Adams County,...,16.3,0.0,442178.0,574216.0,144809.0,8547.0,14844.0,50.0,1924.98,9.7
3246,Biden,North Dakota,5742,ND,0,0.427741,0,northdakota,38000.0,North Dakota,...,0.5,24.8,11349799.0,13099348.0,10527300.0,16495.0,1214201.0,12178.0,69000.80,9.7


### Merging looks good except for the O Brien county
* These are the NAN rows from a left join
* TODO: rename O Brien county in the county facts sheet

In [30]:
merged[merged.isna().any(axis=1)]

,last name,county_x,votes,state,winner,percent_votes,winner_state,county_merge,fips,county_y,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
652,Biden,Satellite caucus 5,71,IA,1,0.559055,0,satellitecaucus5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
653,Sanders,Satellite caucus 5,56,IA,0,0.440945,1,satellitecaucus5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
654,Sanders,Satellite caucus 4,301,IA,1,0.934783,1,satellitecaucus4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
655,Biden,Satellite caucus 4,21,IA,0,0.065217,0,satellitecaucus4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656,Sanders,Satellite caucus 3,1245,IA,1,0.806870,1,satellitecaucus3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657,Biden,Satellite caucus 3,298,IA,0,0.193130,0,satellitecaucus3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,Sanders,Satellite caucus 2,583,IA,1,0.895545,1,satellitecaucus2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,Biden,Satellite caucus 2,68,IA,0,0.104455,0,satellitecaucus2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
660,Sanders,Satellite caucus 1,365,IA,1,0.844907,1,satellitecaucus1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,Biden,Satellite caucus 1,67,IA,0,0.155093,0,satellitecaucus1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge candidate features
* Import the candidate features and merge into the dataframe on the last name

In [54]:
candidate_features = pd.read_excel('data/candidate_features.xlsx')
candidate_features = candidate_features.head(2)
candidate_features.rename(columns={"Candidate": "last name"}, inplace=True)

merged = pd.merge(merged, candidate_features, how='inner', on='last name')

merged

,last name,county_x,votes,state,winner,percent_votes,winner_state,county_merge,fips,county_y,...,carbon_tax,north_korea_meet,free_college,children_public,ban_fracking,border_crossing_penalties,medicare_for_all,private_insurance,eliminate_electoral_college,eliminate_filibuster
0,Biden,Winston,266,AL,1,0.694517,1,winston,1133,Winston County,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Biden,Wilcox,1864,AL,1,0.867784,1,wilcox,1131,Wilcox County,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Biden,Washington,1109,AL,1,0.876680,1,washington,1129,Washington County,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Biden,Walker,1390,AL,1,0.720954,1,walker,1127,Walker County,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,Biden,Tuscaloosa,11825,AL,1,0.769006,1,tuscaloosa,1125,Tuscaloosa County,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3231,Sanders,Chelan,4075,WA,0,0.462963,0,chelan,53007,Chelan County,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3232,Sanders,Benton,6911,WA,0,0.448330,0,benton,53005,Benton County,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3233,Sanders,Asotin,705,WA,0,0.386725,0,asotin,53003,Asotin County,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3234,Sanders,Adams,288,WA,1,0.503497,0,adams,53001,Adams County,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


# Clean up the final dataframe
* Use county_y as county name
* Drop county_x and county_merge

In [3]:
merged.rename(columns={'county_y': 'county'}, inplace=True)
merged.drop(['county_x', 'county_merge'], axis=1, inplace=True)

merged

NameError: name 'merged' is not defined

# Save to csv

In [56]:
merged.to_csv('combined_results_supertues.csv', index=False)